In [ ]:
# imports
import pandas as pd
import numpy as np
import math, itertools
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
from ortools.linear_solver import pywraplp as OR

In [ ]:
clicker_data = pd.read_csv('transportation_caterer.csv', index_col=0)
display(clicker_data)

In [ ]:
def transportation(data, integer=False):
    """A model for solving the transportation problem.
    
    Args:
        data (pd.DataFrame): Dataframe with demand, supplies, and cost matrix.
    """
    ORIG = list(data.index)[:-1]                                # origins
    DEST = list(data.columns)[:-1]                              # destinations
    supply = data['supply'][:-1].to_dict()                      # supply
    demand = data.transpose()['demand'][:-1].to_dict()          # demand
    cost = data.iloc[:-1,:-1].transpose().to_dict()
    cost = {(i,j) : cost[i][j] for i in cost for j in cost[i]}  # cost
    ARCS = list(cost)                                           # arcs
    
    # define model
    m = OR.Solver('transportation', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}  # units to be shipped on each edge
    for i,j in ARCS:
        if integer:
            x[i,j] = m.IntVar(0, m.infinity(), ('(%s, %s)' % (i,j))) 
        else:
            x[i,j] = m.NumVar(0, m.infinity(), ('(%s, %s)' % (i,j)))
        
    # objective function
    m.Minimize(sum(cost[i,j]*x[i,j] for i,j in ARCS))
        
    # subject to: all supply delivered at each origin node
    for i in ORIG:
        m.Add(sum(x[i,j] for j in DEST) == supply[i])
        
    # subject to: demand met at each demand node
    for j in DEST:
        m.Add(sum(x[i,j] for i in ORIG) == demand[j])
    
    return m, x

In [ ]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())

    for var in m.variables():
        if var.solution_value() > 0.0:
            print(var.name(), ':',  var.solution_value())

In [ ]:
m, x = transportation(clicker_data)
solve(m)

In [ ]:
def generate_caterers_df(filename, demands, costs, num_cycles=1, penalty=1000, unused_cost=0):
    """A function to generate a pandas DataFrame for the periodic extension of the caterer's problem.
    
    Args:
        filename (str): file where CSV will be written; must end in .csv
        demands (list): demands for the original demand points*
        costs (list): costs for buying new items from the store supply point or reusing them
                      after a certain number of days**
        num_cycles (int): how many periods to generate (default: 1)
        penalty (int): cost of shipping back in time or outside of constraints (default: 1000)
        unused_cost (int): cost of sending supply to dummy demand node (default: 0)
        
        *Note: demands MUST be a list of nonnegative numbers, and must satisfy
        len(demands) = n, where n is the number of days in a cycle
        
        **Note: costs MUST be formatted as the following: entry 0 is the cost of buying a new item;
        entry k > 0 is the cost of reusing an item after k days
    """
    assert filename[-4:] == ".csv", "File name provided not in CSV format"
    assert not any(float(x) < 0 for x in demands), "Demand values must be nonnegative"
    
    n = len(demands)
    total_days = n*num_cycles # number of demand days
    total_demand = sum(i for i in demands)*num_cycles # total demand
    
    # create labels for demand days
    demand_labels = []
    cycle_no = 1
    for i in range(num_cycles):
        for j in range(1,n+1):
            demand_labels.append((str(j) + ("(%d)" % cycle_no if cycle_no > 1 else "")))
        cycle_no += 1
    
    # dictionary of lists containing elements
    rows = {}
    
    # add row of column headers
    header = [""] + [l for l in demand_labels] + ["d","supply"]
    rows[1] = header

    # add store supply node row
    row2 = ["s"] + ([costs[0]] * total_days) + [unused_cost,total_demand]    
    rows[2] = row2
    
    # add days' end supply node rows
    row_no = 3
    num_reuse = len(costs) - 1
    demand_list = demands * num_cycles
    supply_labels = [i + '\'' for i in demand_labels]
    for i in range(1,total_days):
        row = [supply_labels[i-1]]
        cost_list = []
        
        for j in range(i):
            cost_list.append(penalty)
            
        spaces_left = total_days - i
        k = 0
        while spaces_left > 0:
            if k < num_reuse:
                k += 1
                cost_list.append(costs[k])
            else:
                cost_list.append(penalty)
            spaces_left -= 1
        
        row.extend(cost_list)
        row.extend([unused_cost,demand_list[i-1]])
        
        rows[row_no] = row
        row_no += 1
    
    # add row of demand values
    last_row = ["demand"] + [d for d in demand_list]
    
    dummy_demand = total_demand - demand_list[-1]
    last_row.extend([dummy_demand, total_demand + dummy_demand])
    
    rows[row_no] = last_row

    # create dataframe
    rows_list = []
    header = rows[1]
    for k in range(2,row_no+1):
        row = rows[k]
        dict1 = dict(zip(header,row))
        rows_list.append(dict1)
    
    df = pd.DataFrame(rows_list, columns=header)
    
    # write to CSV
    df.to_csv(filename, encoding='utf-8', index=False)
    
    return pd.read_csv(filename, index_col = 0)

In [ ]:
file_name = 'empty.csv' ## make sure the file specified exists before writing to it!!

In [ ]:
def solve_periodic(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    
    """
    original
        for var in m.variables():
        if var.solution_value() > 0.0:
            print(var.name(), ':',  var.solution_value())
    """
    
    store_nodes = {}
    weeks = {}
    for x in m.variables():
        v = x.solution_value()
        if v > 0.0:
            name = x.name()
            source, sink = tuple(name[1:-1].split(", "))
            if source == 's':
                store_nodes[sink] = v
            else:
                cycle_no = 1
                day_no = 0
                if '(' in source:
                    first = source.find('(')
                    second = source.find(')')
                    cycle_no = int(source[first+1:second])
                    day_no = int(source[:first])
                else:
                    day_no = int(source[:source.index("'")])
                
                ### weeks[cycle_no] = [[day_nos],[values]]
                if cycle_no in weeks:
                    weeks[cycle_no][0].append(day_no)
                    weeks[cycle_no][1].append(v)
                else:
                    weeks[cycle_no] = [[day_no],[v]]
                    
    #print("Store values: " + str(list(store_nodes.values())))
    
    last_unique_days = []
    last_unique_vals = []
    unique = 0
    non_unique = []
    for i in list(weeks):
        days = weeks[i][0]
        vals = weeks[i][1]
        
        if days != last_unique_days or vals != last_unique_vals:
            if non_unique:
                print("Week %d-%d values: same as week %d" % (min(non_unique),max(non_unique),unique))
            print("Week %d values: unique" % i)
            # print(*vals, sep=', ') 
            last_unique_days = days
            last_unique_vals = vals
            unique = i
        else:
            non_unique.append(i)           


In [ ]:
# For caterer's problem input used in lab:
# test = generate_caterers_df(file_name,[15,12,18,6],[20,10,6],num_cycles=1)

test = generate_caterers_df(file_name,[15,12,18,6,13,4,19],[20,10,6],num_cycles=10)

In [ ]:
m,x = transportation(test)
solve_periodic(m)

#### min-cost flow (not periodic yet)

In [ ]:
def caterers_mcf(integer=False):
    """A min-cost flow model for solving the caterer's problem.
    
    Args:
        data (pd.DataFrame): Dataframe with demand, supplies, and cost matrix.
    """
    demands = {1:15,2:12,3:18,4:6}
    costs = {0:20,1:10,2:6}
    
    day_starts = list(demands.keys())              # demand nodes
    day_ends = [-1*i for i in day_starts]   # supply nodes
    first = day_starts[0]
    last = day_starts[-1]
    
    nodes = day_starts + day_ends + ["s","d"]
    
    edges = {} # (i,j) : (LB, UB, cost)
    
    edges["d","s"] = (0, "inf", 0)
    for i in day_starts:
        j = -1*i
        edges[("s",i)] = (0, "inf", costs[0])
        edges[(j,"d")] = (0, "inf", 0)
        edges[(i,j)] = (demands[i], "inf", 0)
        #if i != last:
          #  edges[(i,i+1)] = (0, "inf", 0)
        if i+2 <= last:
            edges[(j,i+2)] = (0, "inf", costs[2])
            if i+1 <= last:
                edges[(j,i+1)] = (0, "inf", costs[1])
        
        
        
    ARCS = edges 
    
    # define model
    m = OR.Solver('min cost flow', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}  # units to be shipped on each edge
    for i,j in ARCS:
        lb, ub, cost = ARCS[i,j]
        if integer:
            x[i,j] = m.IntVar(lb, m.infinity() if ub == "inf" else ub, ('(%s, %s)' % (i,j))) 
        else:
            x[i,j] = m.NumVar(lb, m.infinity() if ub == "inf" else ub, ('(%s, %s)' % (i,j)))
        
    # objective function
    m.Minimize(sum(ARCS[i,j][2]*x[i,j] for i,j in ARCS))
    
    # subject to: flow balance
    for node in nodes:
        m.Add(sum(x[i,j] for i,j in ARCS if node==i) == sum(x[i,j] for i,j in ARCS if node==j))
    
    return m, x

In [ ]:
m,x = caterers_mcf()

In [ ]:
solve(m)

In [ ]:
import csv
def print_dict_to_csv(filename,d):
    assert filename[-4:] == ".csv", "File name provided not in CSV format"
    with open(filename, 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        for r in sorted(list(d.keys())):
            writer.writerow(d[r])                